# FIT5196 Task 1 in Assessment 1
#### Student Name: Armin Berger
#### Student ID: 26255367

Date: 23/08/2020

Version: 1.0

Environment: Python 2.7.11 and Jupyter notebook

Libraries used: 
* re (for regular expression, included in Anaconda Python 2.7)
* os (library used for interacting with operating systems, included in Anaconda Python 2.7) 
* langid (library used for language identification, included in Anaconda Python 2.7) 


# 1.  Import libraries 

Firstly, we will install and import all the required libraries

In [ ]:
# installing langid library, in case it is not installed 
pip install langid

In [ ]:
# importing the three libraries required for this assignment(re, os,and langid)

import re # used for regular expression

import os # used for reading in multiple file from the local os 

import langid # used to check whether a tweet is in english 

# import langid # used for identify only tweets in this case (does language identification in general)

# 2. Check for the path of the current working directory

In [ ]:
# we need to know the location of the current working directory in order to read in the folder containing the
# twitter txt files

# Get the current working directory (cwd) in case it is needed for reading in files
cwd = os.getcwd()

# print current working directory
cwb

# Get all the files in that directory
files = sorted(os.walk('26255367'))

In [ ]:
# check files names that were read in
files[0]

# 3. Parse the txt files, extract  and save the main three data poinst of each tweet

Since the text data is only semi structred we need to find ways in oder to only extract the information we need. This is largely achieved throught the use of regular expressions. For this purpose I came up with three sets of regular expressions to extract the id, text, and the date created.

In [ ]:
## NOTE: this code block runs for a while but creates the desired output (please don't restart it, just let it run)

# we create a set of global data structures to save the processed tweet data
new_tweets = []       # list that holds dicts of tweets

# loop through all files
for entry in files[0][2]:
    
    # we read in the txt data using a combination of the open() and the read() method
    # this generates a large string which is saved as the variable tweets
    # tweets = open('/Users/arminberger/Desktop/Code/fit5196_code/assign_1/26255367_txt/' + entry).read()
    with open('26255367/'+entry, encoding = 'latin-1') as tweets:
        
        # read in the tweet
        tweets = tweets.read()
        
        # we split the large string named tweets into actual individual tweets 
        # since each tweet is enclosed in {} and seperated by a comma , we can use regular expression to split the 
        # large string into individual tweets and save the as list named 'single_tweets'
        single_tweets = re.split(r'},{', tweets)

        # iterate through the single_tweet list dict by dict using a for loop
        for item in single_tweets:

            ## FIRST ##, extract the text
            # valid tweets for xml file must have three elements - the tweet, created date and time, and the user ID. 
            # tweet_dict is created in order to save the tweet and the relevant information 
            tweet_text = re.search(r'"text":"(.+?)"',item)
            tweet_date = re.search(r'"created_at":"(.+?)","id":',item)
            tweet_id = re.search(r'"id":"([0-9]{19})"',item)
            
            # checking that tweet text, date and id is not None
            if tweet_text is not None and tweet_date is not None and tweet_id is not None:
                
                # dict to save individual tweets
                tweet_dict = {}
                
                # get the first match in tweet_text 
                actual_text = tweet_text.group(1)
                
                # if the tweet ends with '\', .decode('unicode_escape') raises an error. 
                # therefore, the backslash at the end of any tweets is eliminated.
                if re.search(r'.*\\$', actual_text) is not None:
                    
                    # remove backslash at the end using .replace()
                    actual_text = actual_text.replace('\\','')

                # the strings with escaped sequence should be corrected into unescaped sequence
                # to decode and encode the surrogate pairs seperately from other escaped sequence, 
                # .decode('unicode_escape') is performed seperately based on whether a string contains either surrogate pairs, 
                # \n or both 
                # if the string contains '\n' with escaped sequence but no emoticons 
                if re.search(r'\\\\n',actual_text) is not None and re.search(r'\\uD', actual_text) is None: 
                    
                    # directly unescape sequence
                    actual_text = actual_text.encode('utf-8').decode('unicode_escape')

                # if the string contains both the emoticons and \n OR the string contains only emoticons
                if (re.search(r'\\\\n',actual_text) is not None and re.search(r'\\uD', actual_text) is not None)\
                or (re.search(r'\\\\n',actual_text) is None and re.search(r'\\uD', actual_text) is not None): 
                
                    # unescape backslash and decode the emoticons 
                    actual_text = actual_text.encode('utf-8').decode('unicode_escape').encode("utf-16", "surrogatepass").decode("utf-16")
                
                # after decoding and encoding the strings, the tweet is written into the dictionary 
                tweet_dict['text']= str(actual_text)
                

                ## SECOND ##, extract the date/time
                actual_date = tweet_date.group(1)
                
                # assign the extracted date to the key 'date' and add it to the tweet_dict
                tweet_dict['date']= str(actual_date)

                ## THIRD ##, extract the ID
                actual_id = tweet_id.group(1)
                
                # then we assign the extracted id to the key id and add it to the tweet_dict
                tweet_dict['id']= str(actual_id)

                # the structured tweet_dict is appended into new_tweets
                new_tweets.append(tweet_dict)

# filter out the non-English tweet using list comprehension 
new_tweets = [dic for dic in new_tweets if dic['text'] !='' and langid.classify(dic['text'])[0]=='en']

# the list of non-duplicate tweets 
unique_tweets = []

# loop through new_tweets
for item in new_tweets: 
    
    # in case tweet is not in unique_tweets
    if item not in unique_tweets: 
        
        # append tweet
        unique_tweets.append(item)

# 4. Save the all unique tweets in an xml format 

In the last step of task one we need to save all the extracted tweets in an xml format.

In [ ]:
# create an xml file to write to
tweet_file = open('<26255367>.xml','w')

# the XML file starts with the version of XML and its character encoding 
# the root tag of the XML file is <data>
# all the tweets are enclosed within this tag 
# the tag 'data' contains <tweets> tag, whose attribute is 'date'
tweet_file.write('''<?xml version="1.0" encoding="UTF-8"?>\n<data>\n''')

# looping through the sorted list of unique_tweets 
# the unique_tweets is sorted based on the created date of every tweet
for i in range(len(sorted(unique_tweets, key = lambda x: x['date']))): 
    
    # extract the created date of the tweet 
    date = unique_tweets[i]['date'].split('T')[0]
    
    if i >0:
        
        # extract the created date of the tweet before this loop 
        former_date = unique_tweets[i-1]['date'].split('T')[0]
        
        # if the created date of the tweet differs from the one before, 
        if date != former_date: 
            
            # a new <tweets> tag is created followed by the date attribute 
            tweet_file.write('''</tweets>\n<tweets date="{}">\n'''.format(date))
    
    else:
        
        # the first tweet of the list unique_tweets does not have any former date to compare 
        # therefore the tweet starts off with a new tag <tweets>
        tweet_file.write('''<tweets date="{}">\n'''.format(date))
    
    # write in the actual tweet text into <tweet> tag, whose attribute is the id of the tweet 
    tweet_file.write('''<tweet id="{}">{}</tweet>\n'''.format(unique_tweets[i]['id'], unique_tweets[i]['text']))

# finish off the XML file by close the last <tweets> tag and the <data> tag 
tweet_file.write('''</tweets>\n</data>''')

# we close the xml file after writing to it
tweet_file.close()

# 5. Summary

In summary, I have been able to read in the provided semi-structed textfile data, extract the required information, and write that information to an xml file. The proceedure can be broken down into the following steps:

- Firstly, we imported the required libraries re, os,and langid
    - re was required in order to use regular expressions
    - os was required to open and write to text and xml files
    - landid was required to be able to determine whether a tweet is in english or not
    
- Secondly, we parsed the text file and turned its content into one long continous string 

- Thridly, we extracted and saved the main three data poinst of each tweet, which are the tweets unique id, its text and it's date/time of creation
    - the extraction of the required information happen in two main steps
    - first, the long string was split into individual tweets and saved as a list of strings
    - second, list of strings was iterated through and within each tweet id, text, and date were extracted with the help of regular expression]
    - fourth, each id, txt, and date data point was added as a value to a dict
    - lastly, it was checked whether a certain tweet has aready been saved earlier. This was done by turning the dict into a tuple and checking whether it was already present in the dict. If that was not the case the dict was appended to the list of tweets 

- Fourthly, we created a new xml file and wrote all the data contained in the tweet list to the xml file


The main finding of this task for me was that, despite every tweet being largely different to another in terms of lenght and the characters it contains, the fact that the tweet data was semi-structered already made it easier to seperate each tweet from another and extract the important information. I learned the importance of looking for repreating patterns within data to guide the wrangleing process.